In [6]:
from pyfaidx import Fasta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Full list of the transcripts

Não faz sentido ir buscar o gene id, 

In [7]:
transcripts = Fasta('Homo_sapiens.GRCh38.cdna.all.fa')

transcript_sequence = pd.DataFrame(transcripts.keys(), columns=["id_version"])
transcript_sequence["Transcript_ID"]= transcript_sequence["id_version"].str.split(".").str.get(0) #

FastaNotFoundError: Cannot read FASTA from file Homo_sapiens.GRCh38.cdna.all.fa

In [5]:
transcript_sequence

NameError: name 'transcript_sequence' is not defined

The dataset Homo_sapiens.GRCh38.cdna.all.fa was downloaded from the Ensembl website, list of all the transcripts and respective sequences  

gene_sequence= Ensemble transcript ID


In [34]:
transcript_sequence

,id_version,Transcript_ID
0,ENST00000415118.1,ENST00000415118
1,ENST00000448914.1,ENST00000448914
2,ENST00000434970.2,ENST00000434970
3,ENST00000631435.1,ENST00000631435
4,ENST00000632684.1,ENST00000632684
...,...,...
207244,ENST00000568156.1,ENST00000568156
207245,ENST00000567218.1,ENST00000567218
207246,ENST00000569984.1,ENST00000569984
207247,ENST00000624828.1,ENST00000624828


# Get the Canonical dataset

In [35]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')


df = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name', 'ensembl_transcript_id', 'transcript_is_canonical'])

Can't extract the NCBI ID here since there are diferent annotations

In [36]:
transcrip_df=df[df["Ensembl Canonical"]==1].copy() # filter to only show the canonical genes

In [37]:
transcrip_df

,Gene stable ID,Gene name,Transcript stable ID,Ensembl Canonical
0,ENSG00000210049,MT-TF,ENST00000387314,1.0
1,ENSG00000211459,MT-RNR1,ENST00000389680,1.0
2,ENSG00000210077,MT-TV,ENST00000387342,1.0
3,ENSG00000210082,MT-RNR2,ENST00000387347,1.0
4,ENSG00000209082,MT-TL1,ENST00000386347,1.0
...,...,...,...,...
276889,ENSG00000236500,CD24P1,ENST00000422383,1.0
276892,ENSG00000197312,DDI2,ENST00000480945,1.0
276898,ENSG00000215695,RSC1A1,ENST00000345034,1.0
276899,ENSG00000271742,NaN,ENST00000606262,1.0


transcrip_df is a dataset of all the human genes with their ensensemble gene IDS, their canonical transcripts and their respective transcript ids.

### Merge the full transcripts with the canonical transcripts

The purpose of this is to extract the Transcript version

In [38]:
cano_df = pd.merge(transcrip_df,transcript_sequence , left_on='Transcript stable ID', right_on='Transcript_ID')

In [39]:
clean_df = cano_df[["Gene name","Gene stable ID","Transcript_ID",'id_version']].copy()

In [40]:
clean_df

,Gene name,Gene stable ID,Transcript_ID,id_version
0,MT-ND1,ENSG00000198888,ENST00000361390,ENST00000361390.2
1,MT-ND2,ENSG00000198763,ENST00000361453,ENST00000361453.3
2,MT-CO1,ENSG00000198804,ENST00000361624,ENST00000361624.2
3,MT-CO2,ENSG00000198712,ENST00000361739,ENST00000361739.1
4,MT-ATP8,ENSG00000228253,ENST00000361851,ENST00000361851.1
...,...,...,...,...
41079,CHCHD2P6,ENSG00000235084,ENST00000454346,ENST00000454346.1
41080,CD24P1,ENSG00000236500,ENST00000422383,ENST00000422383.1
41081,DDI2,ENSG00000197312,ENST00000480945,ENST00000480945.6
41082,RSC1A1,ENSG00000215695,ENST00000345034,ENST00000345034.2


In [41]:
clean_df['Gene stable ID'].value_counts().head(20)

ENSG00000198888    1
ENSG00000204628    1
ENSG00000183770    1
ENSG00000132463    1
ENSG00000007237    1
ENSG00000092148    1
ENSG00000206262    1
ENSG00000157654    1
ENSG00000163497    1
ENSG00000206260    1
ENSG00000184814    1
ENSG00000233701    1
ENSG00000186918    1
ENSG00000251137    1
ENSG00000223452    1
ENSG00000114113    1
ENSG00000284976    1
ENSG00000163499    1
ENSG00000178631    1
ENSG00000169241    1
Name: Gene stable ID, dtype: int64

After removing the NCBI IDS there were no more duplicates

In [42]:
specific_transcript_id = 'ENST00000451548'
rows_with_specific_id = clean_df[clean_df['Transcript_ID'] == specific_transcript_id]
rows_with_specific_id

,Gene name,Gene stable ID,Transcript_ID,id_version
229,TSPY1,ENSG00000258992,ENST00000451548,ENST00000451548.6


## NAN

In [43]:
print(clean_df.isna().sum())

Gene name         6406
Gene stable ID       0
Transcript_ID        0
id_version           0
dtype: int64


# Add the sequences

In [55]:
def get_seq(id):
    if str(id) in transcripts:
                seq=transcripts[id][:].seq
                return seq

In [57]:
clean_dna=clean_df.copy()
clean_dna["DNA_seq"]=clean_dna['id_version'].apply(get_seq)

In [58]:
clean_dna

,Gene name,Gene stable ID,Transcript_ID,id_version,DNA_seq
0,MT-ND1,ENSG00000198888,ENST00000361390,ENST00000361390.2,ATACCCATGGCCAACCTCCTACTCCTCATTGTACCCATTCTAATCG...
1,MT-ND2,ENSG00000198763,ENST00000361453,ENST00000361453.3,ATTAATCCCCTGGCCCAACCCGTCATCTACTCTACCATCTTTGCAG...
2,MT-CO1,ENSG00000198804,ENST00000361624,ENST00000361624.2,ATGTTCGCCGACCGTTGACTATTCTCTACAAACCACAAAGACATTG...
3,MT-CO2,ENSG00000198712,ENST00000361739,ENST00000361739.1,ATGGCACATGCAGCGCAAGTAGGTCTACAAGACGCTACTTCCCCTA...
4,MT-ATP8,ENSG00000228253,ENST00000361851,ENST00000361851.1,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...
...,...,...,...,...,...
41079,CHCHD2P6,ENSG00000235084,ENST00000454346,ENST00000454346.1,GGAAGCCGAAGCCACACCTCCCGCATGGCCCCTCCGGCCAGCCGGG...
41080,CD24P1,ENSG00000236500,ENST00000422383,ENST00000422383.1,GCAATGGTGGACAGGCTCAGGCTGGGGCTGCTGCTTCTGGCACTGC...
41081,DDI2,ENSG00000197312,ENST00000480945,ENST00000480945.6,AGACGGACTCGCAGGCGTGTGGCGGCGGCCGTGCTTGCTAGTGAGG...
41082,RSC1A1,ENSG00000215695,ENST00000345034,ENST00000345034.2,AAGAGAAACCCGAGTTTGAGGACCTTATTTTATTCTACGCTGTTTA...


In [59]:
clean_dna.to_csv("transcript_seq.csv",index= False)